In [1]:
! pip install yfinance langchain_pinecone openai python-dotenv langchain-community sentence_transformers

In [2]:
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
import dotenv
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os

In [ ]:
#https://www.geeksforgeeks.org/get-financial-data-from-yahoo-finance-with-python/
data = yf.Ticker("GOOG")
stock_info = data.info
print(stock_info)

{'address1': '1600 Amphitheatre Parkway', 'city': 'Mountain View', 'state': 'CA', 'zip': '94043', 'country': 'United States', 'phone': '650 253 0000', 'website': 'https://abc.xyz', 'industry': 'Internet Content & Information', 'industryKey': 'internet-content-information', 'industryDisp': 'Internet Content & Information', 'sector': 'Communication Services', 'sectorKey': 'communication-services', 'sectorDisp': 'Communication Services', 'longBusinessSummary': 'Alphabet Inc. offers various products and platforms in the United States, Europe, the Middle East, Africa, the Asia-Pacific, Canada, and Latin America. It operates through Google Services, Google Cloud, and Other Bets segments. The Google Services segment provides products and services, including ads, Android, Chrome, devices, Gmail, Google Drive, Google Maps, Google Photos, Google Play, Search, and YouTube. It is also involved in the sale of apps and in-app purchases and digital content in the Google Play and YouTube; and devices,

In [ ]:

GetInformation = yf.Ticker("META")

# get all key value pairs that are available
for key, value in GetInformation.info.items():
	print(key, ":", value)


address1 : 1 Meta Way
city : Menlo Park
state : CA
zip : 94025
country : United States
phone : 650 543 4800
website : https://investor.fb.com
industry : Internet Content & Information
industryKey : internet-content-information
industryDisp : Internet Content & Information
sector : Communication Services
sectorKey : communication-services
sectorDisp : Communication Services
longBusinessSummary : Meta Platforms, Inc. engages in the development of products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality headsets, and wearables worldwide. It operates in two segments, Family of Apps and Reality Labs. The Family of Apps segment offers Facebook, which enables people to share, discuss, discover, and connect with interests; Instagram, a community for sharing photos, videos, and private messages, as well as feed, stories, reels, video, live, and shops; Messenger, a messaging application for people to connect with friends

In [3]:
def get_stock_info(symbol: str) -> dict:
    """
    Retrieves and formats detailed information about a stock from Yahoo Finance.

    Args:
        symbol (str): The stock ticker symbol to look up.

    Returns:
        dict: A dictionary containing detailed stock information, including ticker, name,
              business summary, city, state, country, industry, and sector.
    """
    data = yf.Ticker(symbol)
    stock_info = data.info

    properties = {
        "Ticker": stock_info.get('symbol', 'Information not available'),
        'Name': stock_info.get('longName', 'Information not available'),
        'Business Summary': stock_info.get('longBusinessSummary'),
        'City': stock_info.get('city', 'Information not available'),
        'State': stock_info.get('state', 'Information not available'),
        'Country': stock_info.get('country', 'Information not available'),
        'Industry': stock_info.get('industry', 'Information not available'),
        'Sector': stock_info.get('sector', 'Information not available'),
        "MarketCap": stock_info.get('marketCap', 'Information not available'),
        "Volume": stock_info.get('volume', 'Information not available'),
        'YearBorn': stock_info.get('yearBorn', 'Information not available'),
        'Exchange': stock_info.get('exchange', 'Information not available'),
        "Earnings Quarterly Growth": stock_info.get('earningsQuarterlyGrowth', 'Information not available'),
        "Revenue Per Share": stock_info.get('revenuePerShare', 'Information not available'),
        "Revenue Growth": stock_info.get('revenueGrowth', 'Information not available'),
        "Gross Margins": stock_info.get('grossMargins', 'Information not available'),
        "EBITDA Margins": stock_info.get('ebitdaMargins', 'Information not available'),
        "EBITDA": stock_info.get('ebitda', 'Information not available'),
        "52 Week Change": stock_info.get('52WeekChange', 'Information not available'),
        "website": stock_info.get('website', 'Information not available')
    }

    return properties

In [ ]:
get_stock_info("GOOG")

{'Ticker': 'GOOG',
 'Name': 'Alphabet Inc.',
 'Business Summary': 'Alphabet Inc. offers various products and platforms in the United States, Europe, the Middle East, Africa, the Asia-Pacific, Canada, and Latin America. It operates through Google Services, Google Cloud, and Other Bets segments. The Google Services segment provides products and services, including ads, Android, Chrome, devices, Gmail, Google Drive, Google Maps, Google Photos, Google Play, Search, and YouTube. It is also involved in the sale of apps and in-app purchases and digital content in the Google Play and YouTube; and devices, as well as in the provision of YouTube consumer subscription services. The Google Cloud segment offers infrastructure, cybersecurity, databases, analytics, AI, and other services; Google Workspace that include cloud-based communication and collaboration tools for enterprises, such as Gmail, Docs, Drive, Calendar, and Meet; and other services for enterprise customers. The Other Bets segment se

In [ ]:
def get_company_tickers():
    """
    Downloads and parses the Stock ticker symbols from the GitHub-hosted SEC company tickers JSON file.

    Returns:
        dict: A dictionary containing company tickers and related information.

    Notes:
        The data is sourced from the official SEC website via a GitHub repository:
        https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json
    """
    # URL to fetch the raw JSON file from GitHub
    url = "https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json"

    # Making a GET request to the URL
    response = requests.get(url)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parse the JSON content directly
        company_tickers = json.loads(response.content.decode('utf-8'))

        # Optionally save the content to a local file for future use
        with open("company_tickers.json", "w", encoding="utf-8") as file:
            json.dump(company_tickers, file, indent=4)

        print("File downloaded successfully and saved as 'company_tickers.json'")
        return company_tickers
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

company_tickers = get_company_tickers()

File downloaded successfully and saved as 'company_tickers.json'


In [4]:
import json

def get_company_tickers_from_local_file(file_path):
    """
    Reads and parses the Stock ticker symbols from a locally saved JSON file.

    Args:
        file_path (str): Path to the JSON file in the Colab notebook folder.

    Returns:
        dict: A dictionary containing company tickers and related information.
    """
    try:
        # Open the local JSON file and load its content
        with open(file_path, "r", encoding="utf-8") as file:
            company_tickers = json.load(file)

        print("File loaded successfully from local path.")
        return company_tickers
    except FileNotFoundError:
        print(f"File not found at path: {file_path}")
        return None
    except json.JSONDecodeError:
        print("Error decoding JSON file. Please check the file format.")
        return None

# Path to the JSON file in the Colab notebook folder
file_path = "/content/tickers.json"

# Call the function to load tickers
company_tickers = get_company_tickers_from_local_file(file_path)

# Example usage of the loaded data
if company_tickers:
    print(f"Loaded {len(company_tickers)} company tickers.")


File loaded successfully from local path.
Loaded 1948 company tickers.


In [5]:
len(company_tickers)

1948

In [5]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for the given text using a specified Hugging Face model.

    Args:
        text (str): The input text to generate embeddings for.
        model_name (str): The name of the Hugging Face model to use.
                          Defaults to "sentence-transformers/all-mpnet-base-v2".

    Returns:
        np.ndarray: The generated embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [ ]:
def cosine_similarity_between_sentences(sentence1, sentence2):
    """
    Calculates the cosine similarity between two sentences.

    Args:
        sentence1 (str): The first sentence for similarity comparison.
        sentence2 (str): The second sentence for similarity comparison.

    Returns:
        float: The cosine similarity score between the two sentences,
               ranging from -1 (completely opposite) to 1 (identical).

    Notes:
        Prints the similarity score to the console in a formatted string.
    """
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    similarity_score = similarity[0][0]
    print(f"Cosine similarity between the two sentences: {similarity_score:.4f}")
    return similarity_score


# Example usage
sentence1 = "I like eating cake"
sentence2 = "I like eating sweets"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Cosine similarity between the two sentences: 0.7243


In [6]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "stocks2"
namespace = "stock-descriptions"

hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

<ipython-input-6-aeb05a1383ec>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings()
<ipython-input-6-aeb05a1383ec>:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hu

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
import time
import concurrent.futures

# Initialize tracking lists
successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")


def fetch_with_backoff(stock_ticker: str, retries=3, base_delay=1, multiplier=5):
    """
    Fetches stock data with exponential backoff for handling rate limits.

    Args:
        stock_ticker (str): The stock ticker symbol to fetch data for.
        retries (int): Number of retry attempts for rate-limit errors.
        base_delay (int): Initial delay in seconds for backoff.
        multiplier (int): Factor by which to multiply the delay after each failed attempt.

    Returns:
        dict: The fetched stock data.

    Raises:
        ValueError: If no response is received after retries.
    """
    delay = base_delay  # Start with the base delay
    for attempt in range(retries):
        try:
            # Fetch stock data
            stock_data = get_stock_info(stock_ticker)

            # Check for empty/malformed response
            if not stock_data or not isinstance(stock_data, dict):
                raise ValueError(f"Malformed or empty response for {stock_ticker}")
            return stock_data
        except Exception as e:
            if "Too Many Requests" in str(e):
                print(f"Rate limit hit for {stock_ticker}. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= multiplier  # Larger exponential backoff
            else:
                print(f"Non-retryable error for {stock_ticker}: {e}")
                raise e  # Reraise other exceptions
    # Exhausted retries
    raise ValueError(f"Failed to fetch data for {stock_ticker} after {retries} retries.")


def process_stock(stock_ticker: str) -> str:
    """
    Processes a single stock ticker by fetching its data, generating embeddings,
    and storing the data in Pinecone.

    Args:
        stock_ticker (str): The stock ticker symbol to process.

    Returns:
        str: Success or error message for the stock ticker.
    """
    if stock_ticker in successful_tickers:
        return f"Already processed {stock_ticker}"

    try:
        stock_data = fetch_with_backoff(stock_ticker)

        stock_description = stock_data.get('Business Summary', None)
        if not stock_description:
            raise ValueError(f"Missing Business Summary for {stock_ticker}")

        PineconeVectorStore.from_documents(
            documents=[Document(page_content=stock_description, metadata=stock_data)],
            embedding=hf_embeddings,
            index_name=index_name,
            namespace=namespace
        )

        with open('successful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        successful_tickers.append(stock_ticker)

        return f"Processed {stock_ticker} successfully"

    except Exception as e:
        print(f"Error processing {stock_ticker}: {e}")
        unsuccessful_tickers.append(stock_ticker)

        if len(unsuccessful_tickers) % 10 == 0:
            with open('unsuccessful_tickers.txt', 'a') as f:
                f.write('\n'.join(unsuccessful_tickers[-10:]) + '\n')
        return f"ERROR processing {stock_ticker}: {e}"


def parallel_process_stocks(tickers: list, max_workers: int = 5, batch_delay=5) -> None:
    """
    Processes stock tickers in parallel with dynamic delays to avoid rate limits.

    Args:
        tickers (list): List of stock tickers to process.
        max_workers (int): Maximum number of parallel workers.
        batch_delay (int): Delay in seconds between batches.
    """
    tickers_to_process = [ticker for ticker in tickers if ticker not in successful_tickers]

    if not tickers_to_process:
        print("All tickers are already processed!")
        return

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {
            executor.submit(process_stock, ticker): ticker
            for ticker in tickers_to_process
        }

        for i, future in enumerate(concurrent.futures.as_completed(future_to_ticker)):
            ticker = future_to_ticker[future]
            try:
                result = future.result()
                print(result)
            except Exception as exc:
                print(f"{ticker} generated an exception: {exc}")
                unsuccessful_tickers.append(ticker)

            if (i + 1) % max_workers == 0:
                print(f"Processed {i + 1} tickers. Adding a {batch_delay}-second delay...")
                time.sleep(batch_delay)

    if unsuccessful_tickers:
        with open('unsuccessful_tickers.txt', 'a') as f:
            f.write('\n'.join(unsuccessful_tickers) + '\n')
    print("Processing complete. Check successful_tickers.txt and unsuccessful_tickers.txt for details.")


# Prepare your tickers for processing
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

# Start processing the tickers
parallel_process_stocks(tickers_to_process, max_workers=5)


No existing successful tickers file found
No existing unsuccessful tickers file found
Processed OMVJF successfully
Processed SGSOF successfully
Processed Z successfully
Processed NRSCF successfully
Processed NWS successfully
Processed 5 tickers. Adding a 5-second delay...
Processed SPXSF successfully
Processed RYLPF successfully
Processed PPERF successfully
Processed AMCCF successfully
Processed JHIUF successfully
Processed 10 tickers. Adding a 5-second delay...
Processed KEY-PJ successfully
Processed BIO-B successfully
Processed CHEAF successfully
Processed CHKIF successfully
Processed FAXXF successfully
Processed 15 tickers. Adding a 5-second delay...
Error processing NCSYF: Missing Business Summary for NCSYF
Processed ADTTF successfully
Processed FXFLF successfully
Processed KEY-PK successfully
Processed KKPNF successfully
Processed HLBZF successfully
Processed 20 tickers. Adding a 5-second delay...
Processed EBR-B successfully
Processed CRERF successfully
Processed TEVJF successful

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ADZCF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ADZCF&crumb=S6yynZlJUkP


Error processing ADZCF: Missing Business Summary for ADZCF
Error processing CTSUF: Missing Business Summary for CTSUF
Error processing CTSWF: Missing Business Summary for CTSWF
Error processing CAPNU: Missing Business Summary for CAPNU
Error processing CAPNR: Missing Business Summary for CAPNR
Error processing FRSPF: Missing Business Summary for FRSPF
Error processing SAT: Missing Business Summary for SAT
Processed BHR-PB successfully
ERROR processing HAWEN: Missing Business Summary for HAWEN
Processed CNTHN successfully
Processed AHT-PF successfully
Processed SLNCF successfully
Processed 365 tickers. Adding a 5-second delay...
Error processing SAY: Missing Business Summary for SAY
Error processing SAJ: Missing Business Summary for SAJ
Error processing SAZ: Missing Business Summary for SAZ
Error processing GRND-WT: Missing Business Summary for GRND-WT
Processed NVNXF successfully
Processed EXROF successfully
ERROR processing APLMW: Missing Business Summary for APLMW
Processed AHT-PG su

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SEATW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SEATW&crumb=Edge%3A+Too+Many+Requests


Non-retryable error for IRRXU: Expecting value: line 1 column 1 (char 0)
Error processing IRRXU: Expecting value: line 1 column 1 (char 0)
Non-retryable error for DESLF: Expecting value: line 1 column 1 (char 0)
Error processing DESLF: Expecting value: line 1 column 1 (char 0)
Non-retryable error for CSWCZ: Expecting value: line 1 column 1 (char 0)
Error processing CSWCZ: Expecting value: line 1 column 1 (char 0)
Non-retryable error for IRRXW: Expecting value: line 1 column 1 (char 0)
Error processing IRRXW: Expecting value: line 1 column 1 (char 0)
Non-retryable error for SEATW: Expecting value: line 1 column 1 (char 0)Non-retryable error for SKILW: Expecting value: line 1 column 1 (char 0)
Error processing SKILW: Expecting value: line 1 column 1 (char 0)

Error processing SEATW: Expecting value: line 1 column 1 (char 0)
Non-retryable error for NAMSW: Expecting value: line 1 column 1 (char 0)
Error processing NAMSW: Expecting value: line 1 column 1 (char 0)


...

Non-retryable error for MTB-PH: Expecting value: line 1 column 1 (char 0)
Error processing MTB-PH: Expecting value: line 1 column 1 (char 0)
Non-retryable error for ACZT: Expecting value: line 1 column 1 (char 0)
Error processing ACZT: Expecting value: line 1 column 1 (char 0)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CELG-RI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CELG-RI&crumb=Edge%3A+Too+Many+Requests


Non-retryable error for YHNAU: Expecting value: line 1 column 1 (char 0)
Error processing YHNAU: Expecting value: line 1 column 1 (char 0)
Non-retryable error for YHNAR: Expecting value: line 1 column 1 (char 0)
Error processing YHNAR: Expecting value: line 1 column 1 (char 0)
Non-retryable error for BACQU: Expecting value: line 1 column 1 (char 0)
Error processing BACQU: Expecting value: line 1 column 1 (char 0)
Non-retryable error for CYTHW: Expecting value: line 1 column 1 (char 0)
Error processing CYTHW: Expecting value: line 1 column 1 (char 0)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FCCID?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FCCID&crumb=Edge%3A+Too+Many+Requests


Non-retryable error for CELG-RI: Expecting value: line 1 column 1 (char 0)
Error processing CELG-RI: Expecting value: line 1 column 1 (char 0)
Non-retryable error for UUGWF: Expecting value: line 1 column 1 (char 0)
Error processing UUGWF: Expecting value: line 1 column 1 (char 0)
Non-retryable error for MKDWW: Expecting value: line 1 column 1 (char 0)
Error processing MKDWW: Expecting value: line 1 column 1 (char 0)
Non-retryable error for RMXI: Expecting value: line 1 column 1 (char 0)
Error processing RMXI: Expecting value: line 1 column 1 (char 0)
Non-retryable error for FCCID: Expecting value: line 1 column 1 (char 0)
Error processing FCCID: Expecting value: line 1 column 1 (char 0)
Non-retryable error for ANZGF: Expecting value: line 1 column 1 (char 0)
Error processing ANZGF: Expecting value: line 1 column 1 (char 0)
Processed AFIIQ successfully
Processed VRAYQ successfully
ERROR processing NMTRQ: Missing Business Summary for NMTRQ
Processed BZRD successfully
Processed JUVAF suc